In [1]:
import pandas as pd


In [2]:
vacances = [
    (pd.to_datetime("01/01/2023", format='%d/%m/%Y'), pd.to_datetime("03/01/2023", format='%d/%m/%Y')),
    (pd.to_datetime("18/02/2023", format='%d/%m/%Y'), pd.to_datetime("06/03/2023", format='%d/%m/%Y')),
    (pd.to_datetime("22/04/2023", format='%d/%m/%Y'), pd.to_datetime("09/05/2023", format='%d/%m/%Y')),
    (pd.to_datetime("08/07/2023", format='%d/%m/%Y'), pd.to_datetime("31/12/2023", format='%d/%m/%Y'))
]

jours_feries = ["01/01/2023", "10/04/2023", "01/05/2023", "08/05/2023", "18/05/2023", "19/05/2023", "29/05/2023"]

def is_vacances(date):
    for debut, fin in vacances:
        if debut <= date <= fin:
            return True
    return False

def is_jours_feries(date):
    return date.strftime('%d/%m/%Y') in jours_feries

def is_johv(date):
    return date.weekday() < 5 and not is_vacances(date) and not is_jours_feries(date)

def is_sahv(date):
    return date.weekday() == 5 and not is_vacances(date) and not is_jours_feries(date)

def is_jovs(date):
    return date.weekday() < 5 and is_vacances(date)

def is_savs(date):
    return date.weekday() == 5 and is_vacances(date)

def is_dijfp(date):
    return date.weekday() == 6 or is_jours_feries(date)


In [3]:
df_nb = pd.read_csv('validations-nombre-2023-S1.csv', sep=';')
df_ph = pd.read_csv('validations-profils-2023-S1.csv', sep=';')
df_geo = pd.read_csv('emplacement-des-gares-idf.csv', sep=';', usecols=['Geo Point', 'id_ref_ZdC'])

df_nb['JOUR'] = pd.to_datetime(df_nb['JOUR'], format='%Y-%m-%d')
df_nb['LIBELLE_ARRET'] = df_nb['LIBELLE_ARRET'].str.strip()
df_nb['CAT_JOUR'] = df_nb['JOUR'].apply(lambda x: 'JOHV' if is_johv(x) else 'SAHV' if is_sahv(x) else 'JOVS' if is_jovs(x) else 'SAVS' if is_savs(x) else 'DIJFP')

df_ph['LIBELLE_ARRET'] = df_ph['LIBELLE_ARRET'].str.strip()
df_ph = df_ph[df_ph['TRNC_HORR_60'] != 'ND']
df_ph['TRNC_HORR_60'] = df_ph['TRNC_HORR_60'].str.split('-').str[0].str.split('H').str[0].astype(int)
df_ph = df_ph.rename(columns={'TRNC_HORR_60': 'HEURE'})

df_geo['LATITUDE'] = df_geo['Geo Point'].str.split(',').str[0].astype(float)
df_geo['LONGITUDE'] = df_geo['Geo Point'].str.split(',').str[1].astype(float)
df_geo = df_geo.drop(columns=['Geo Point'])

In [4]:
# Compute the mean NB_VALID for each LIBELLE_ARRET and CAT_JOUR
df_mean = df_nb.groupby(['LIBELLE_ARRET', 'CAT_JOUR'])['NB_VALD'].mean().reset_index()
df_mean = df_mean.pivot(index='LIBELLE_ARRET', columns='CAT_JOUR', values='NB_VALD').reset_index()
df_mean = df_mean.melt(id_vars=['LIBELLE_ARRET'], var_name='CAT_JOUR', value_name='NB_VALD')


In [5]:
# Merge the df_mean_melted with df_ph where the rows "LIBELLE_ARRET" and "CAT_JOUR" are the same
df = pd.merge(df_mean, df_ph, on=['LIBELLE_ARRET', 'CAT_JOUR'], how='inner')

# In each row, multiply pourc_validations by NB_VALID
df['NB_VALD'] = df['NB_VALD'] * df['pourc_validations']

In [6]:
# Drop rows where lda = 999999
df = df[df['lda'] != 999999]

fix_lda = {
    73792: 478926,
    71686: 71697,
    71743: 463564,
    73652: 71607,
    71219: 473829,
    73615: 71359,
    73794: 474151,
    425819: 66915,
    59577: 478855,
    71282: 479068,
    72219: 72225,
    62737: 478505,
    74000: 71434,
    71245: 71229,
    73616: 478885,
    74040: 71139,
    412697: 479919,
    67747: 462934,
    72059: 478883,
    73795: 474152,
    69531: 463754,
    63650: 463850,
    74371: 463843
}

# Replace lda by the correct value
df['lda'] = df['lda'].replace(fix_lda)

df_geo_unique = df_geo.drop_duplicates(subset=['id_ref_ZdC'], keep='first')
df = df.merge(df_geo_unique, left_on='lda', right_on='id_ref_ZdC', how='left')

In [7]:
df = df.drop(columns=['CODE_STIF_TRNS', 'CODE_STIF_RES', 'CODE_STIF_ARRET', 'id_ref_ZdC'])

In [8]:
df['VACANCES'] = df['CAT_JOUR'].apply(lambda x: x in ['SAVS', 'JOVS'])
df['JOUR_OUVRE'] = df['CAT_JOUR'].apply(lambda x: x in ['JOVH', 'JOVS'])
df['SAMEDI'] = df['CAT_JOUR'].apply(lambda x: x in ['SAVH', 'SAVS'])
df['DIMANCHE_JOUR_FERIE'] = df['CAT_JOUR'].apply(lambda x: x == 'DIJFP')


In [9]:
df.to_csv('validations-2023-S1.csv', index=False)